# Pangeo-Enabled ESM Pattern Scaling (PEEPS)
# libraries and helper functions

For simplicity in this notebook, many of the repeated calculations used for
linear pattern scaling are included as functions in the `helper.py` script,
which is a part of this repository and sourced:

In [ ]:
# Load all of the libraries
from matplotlib import pyplot as plt
import pandas as pd


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

# easier to read displays in console
pd.set_option('display.max_columns', None)

# source the helper functions
%run helpers.py

# Intro
We will pull CMIP6 netcdf data from Pangeo (
https://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/).
These will include:
- monthly `tas` data from which to calculate `tgav`
- any specified variables of interest to pattern  scale

# Data


In [ ]:
fetch_pangeo_table().to_csv('pangeo_table.csv', index=False)
# in same directory as this notebook, easy
pangeo_data = pd.read_csv('pangeo_table.csv')


esm_list = pangeo_data.model.unique().copy()
print(esm_list)

# Specify the scenarios and variables of interest for scaling
exp_list = ['historical', 'ssp370',
            'ssp585', 'ssp126', 'ssp245',
            'ssp119', 'ssp460', 'ssp434'  ]

var_list = ['tas', 'pr', 'hurs']

# For a specific esm, variable, scenario, pattern scale

## prep ensemble average data for training
For linear pattern scaling, we fit the regression on the ensemble mean for each
ESM, variable, and scenario. This calculation is included in `helpers.py > do_ps()`.



## loop for annual patterns

In [ ]:
OUTPUT_DIR = 'outputs/'

for esm in esm_list:
    for scn in exp_list:
        for variable in var_list:

            # annual patterns
            savename = (esm + '_' + scn + '_' + variable + '_annual_pattern.nc' )
            print(savename)
            try:
                ensemble_ds = do_ps(esm_name = esm,
                                    var_name = variable,
                                    exp_name  = scn,
                                    monthly_or_annual = 'annual',
                                    fit_intercept = True,
                                    save_resids = True,
                                    tgav_DIR = (OUTPUT_DIR + 'resids_tgavs'))

                if not ensemble_ds == None:
                    ensemble_ds[0].to_netcdf(OUTPUT_DIR + savename)
                    ensemble_ds[1].to_netcdf(OUTPUT_DIR + '/resids_tgavs/' + esm + '_' + scn +
                                             '_' + variable + '_annual_pattern_resids.nc' )

                del(ensemble_ds)




            except:
                print('issue creating ' + savename)
                print('Specific print statement above if data does not exist in pangeo_data.csv.')
                print('And this exception would not have triggered.')
                print('Bigger issue to figure out.')



        # end for over variable
    #end for over experiment
# end for over esm

HadGEM3-GC31-MM_historical_tas_annual_pattern.nc
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f3/Amon/tas/gn/v20191207/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r2i1p1f3/Amon/tas/gn/v20191218/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r3i1p1f3/Amon/tas/gn/v20200601/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r4i1p1f3/Amon/tas/gn/v20200601/
tgav calculation complete, doing scaling with call to reshape_and_scale.
HadGEM3-GC31-MM_historical_pr_annual_pattern.nc
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f3/Amon/pr/gn/v20191207/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r2i1p1f3/Amon/pr/gn/v20191218/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r3i1p1f3/Amon/pr/gn/v20200601/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r4i1p1f3/Amon/pr/gn/v20200601/
non tas var being scaled, must read in tas info to get Tgav for scaling.
Doing now:.
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f

## loop for monthly patterns

In [3]:
months = pd.DataFrame(data={
            'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            'month_name': ['jan', 'feb', 'mar', 'apr',
                           'may', 'jun', 'jul', 'aug',
                           'sep', 'oct', 'nov', 'dec']
        })

OUTPUT_DIR = 'outputs/'

for esm in esm_list:
    for scn in exp_list:
        for variable in var_list:

            savename = (esm + '_' + scn + '_' + variable + '_monthly_patterns' )
            print(savename)
            try:
                ensemble_ds = do_ps(esm_name = esm,
                                    var_name = variable,
                                    exp_name  = scn,
                                    monthly_or_annual = 'monthly',
                                    fit_intercept = True,
                                    save_resids=True)



                if not ensemble_ds == None:
                    for ind in range(1, 13):
                        month_nm = months.loc[(ind - 1), 'month_name']
                        ensemble_ds[(ind-1)][0].to_netcdf(OUTPUT_DIR + savename +
                                                          '_' + month_nm + '.nc')
                        ensemble_ds[(ind-1)][1].to_netcdf(OUTPUT_DIR + 'resids_tgavs/' +savename +
                                                          '_' + month_nm + '_resids.nc')

                del(ensemble_ds)




            except:
                print('issue creating ' + savename)
                print('Specific print statement above if data does not exist in pangeo_data.csv.')
                print('And this exception would not have triggered.')
                print('Bigger issue to figure out.')



        # end for over variable
    #end for over experiment
# end for over esm

HadGEM3-GC31-MM_historical_tas_monthly_patterns
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f3/Amon/tas/gn/v20191207/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r2i1p1f3/Amon/tas/gn/v20191218/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r3i1p1f3/Amon/tas/gn/v20200601/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r4i1p1f3/Amon/tas/gn/v20200601/
tgav calculation complete, doing scaling with call to reshape_and_scale.
HadGEM3-GC31-MM_historical_pr_monthly_patterns
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f3/Amon/pr/gn/v20191207/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r2i1p1f3/Amon/pr/gn/v20191218/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r3i1p1f3/Amon/pr/gn/v20200601/
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r4i1p1f3/Amon/pr/gn/v20200601/
non tas var being scaled, must read in tas info to get Tgav for scaling.
Doing now:.
gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-MM/historical/r1i1p1f3/

This combination of data: 
GFDL-ESM4 pr ssp126
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp245_tas_monthly_patterns
This combination of data: 
GFDL-ESM4 tas ssp245
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp245_pr_monthly_patterns
This combination of data: 
GFDL-ESM4 pr ssp245
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp119_tas_monthly_patterns
This combination of data: 
GFDL-ESM4 tas ssp119
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp119_pr_monthly_patterns
This combination of data: 
GFDL-ESM4 pr ssp119
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp460_tas_monthly_patterns
This combination of data: 
GFDL-ESM4 tas ssp460
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp460_pr_monthly_patterns
This combination of data: 
GFDL-ESM4 pr ssp460
does not exist in pangeo_data.csv. Skipping.
GFDL-ESM4_ssp434_tas_monthly_patterns
This combination of data: 
GFDL-ESM4 tas ssp434
does not exist in pangeo_data.csv. Skippin

This combination of data: 
IPSL-CM6A-LR pr ssp119
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR_ssp460_tas_monthly_patterns
This combination of data: 
IPSL-CM6A-LR tas ssp460
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR_ssp460_pr_monthly_patterns
This combination of data: 
IPSL-CM6A-LR pr ssp460
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR_ssp434_tas_monthly_patterns
This combination of data: 
IPSL-CM6A-LR tas ssp434
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR_ssp434_pr_monthly_patterns
This combination of data: 
IPSL-CM6A-LR pr ssp434
does not exist in pangeo_data.csv. Skipping.
CNRM-CM6-1_historical_tas_monthly_patterns
This combination of data: 
CNRM-CM6-1 tas historical
does not exist in pangeo_data.csv. Skipping.
CNRM-CM6-1_historical_pr_monthly_patterns
This combination of data: 
CNRM-CM6-1 pr historical
does not exist in pangeo_data.csv. Skipping.
CNRM-CM6-1_ssp370_tas_monthly_patterns
This combination of data: 
CNRM-CM6-1 tas 

gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f3/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r2i1p1f1/Amon/pr/gn/v20180827/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r2i1p1f2/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r2i1p1f3/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r3i1p1f1/Amon/pr/gn/v20180828/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r3i1p1f2/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r3i1p1f3/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r4i1p1f1/Amon/pr/gn/v20180828/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r4i1p1f2/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r4i1p1f3/Amon/pr/gn/v20190903/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r5i1p1f1/Amon/pr/gn/v20180828/
gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical

GISS-E2-1-G_ssp585_pr_monthly_patterns
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r2i1p1f2/Amon/pr/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r3i1p1f2/Amon/pr/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r4i1p1f2/Amon/pr/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r5i1p1f2/Amon/pr/gn/v20200115/
non tas var being scaled, must read in tas info to get Tgav for scaling.
Doing now:.
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r2i1p1f2/Amon/tas/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r3i1p1f2/Amon/tas/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r4i1p1f2/Amon/tas/gn/v20200115/
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r5i1p1f2/Amon/tas/gn/v20200115/
tgav calculation complete, doing scaling with call to reshape_and_scale.
GISS-E2-1-G_ssp126_tas_monthly_patterns
gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-

...

This combination of data: 
CMCC-CM2-HR4 tas ssp126
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp126_pr_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 pr ssp126
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp245_tas_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 tas ssp245
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp245_pr_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 pr ssp245
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp119_tas_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 tas ssp119
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp119_pr_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 pr ssp119
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp460_tas_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 tas ssp460
does not exist in pangeo_data.csv. Skipping.
CMCC-CM2-HR4_ssp460_pr_monthly_patterns
This combination of data: 
CMCC-CM2-HR4 pr ssp46

CAS-ESM2-0_ssp370_pr_monthly_patterns
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r1i1p1f1/Amon/pr/gn/v20201228/
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r3i1p1f1/Amon/pr/gn/v20201228/
non tas var being scaled, must read in tas info to get Tgav for scaling.
Doing now:.
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r1i1p1f1/Amon/tas/gn/v20201228/
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp370/r3i1p1f1/Amon/tas/gn/v20201228/
tgav calculation complete, doing scaling with call to reshape_and_scale.
CAS-ESM2-0_ssp585_tas_monthly_patterns
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r1i1p1f1/Amon/tas/gn/v20201228/
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r3i1p1f1/Amon/tas/gn/v20201228/
tgav calculation complete, doing scaling with call to reshape_and_scale.
CAS-ESM2-0_ssp585_pr_monthly_patterns
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r1i1p1f1/Amon/pr/gn/v20201228/
gs://cmip6/CMIP6/ScenarioMIP/CAS/CAS-ESM2-0/ssp585/r3i1p1f1/Amon/pr/gn/v2020

This combination of data: 
EC-Earth3-CC tas ssp460
does not exist in pangeo_data.csv. Skipping.
EC-Earth3-CC_ssp460_pr_monthly_patterns
This combination of data: 
EC-Earth3-CC pr ssp460
does not exist in pangeo_data.csv. Skipping.
EC-Earth3-CC_ssp434_tas_monthly_patterns
This combination of data: 
EC-Earth3-CC tas ssp434
does not exist in pangeo_data.csv. Skipping.
EC-Earth3-CC_ssp434_pr_monthly_patterns
This combination of data: 
EC-Earth3-CC pr ssp434
does not exist in pangeo_data.csv. Skipping.
CMCC-ESM2_historical_tas_monthly_patterns
gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/Amon/tas/gn/v20210114/
tgav calculation complete, doing scaling with call to reshape_and_scale.
CMCC-ESM2_historical_pr_monthly_patterns
gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/Amon/pr/gn/v20210114/
non tas var being scaled, must read in tas info to get Tgav for scaling.
Doing now:.
gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/Amon/tas/gn/v20210114/
tgav calculation 

This combination of data: 
MIROC-ES2H pr ssp126
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp245_tas_monthly_patterns
This combination of data: 
MIROC-ES2H tas ssp245
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp245_pr_monthly_patterns
This combination of data: 
MIROC-ES2H pr ssp245
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp119_tas_monthly_patterns
This combination of data: 
MIROC-ES2H tas ssp119
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp119_pr_monthly_patterns
This combination of data: 
MIROC-ES2H pr ssp119
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp460_tas_monthly_patterns
This combination of data: 
MIROC-ES2H tas ssp460
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp460_pr_monthly_patterns
This combination of data: 
MIROC-ES2H pr ssp460
does not exist in pangeo_data.csv. Skipping.
MIROC-ES2H_ssp434_tas_monthly_patterns
This combination of data: 
MIROC-ES2H tas ssp434
does not exist in pangeo_da

This combination of data: 
IPSL-CM6A-LR-INCA tas ssp370
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp370_pr_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA pr ssp370
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp585_tas_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA tas ssp585
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp585_pr_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA pr ssp585
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp126_tas_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA tas ssp126
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp126_pr_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA pr ssp126
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp245_tas_monthly_patterns
This combination of data: 
IPSL-CM6A-LR-INCA tas ssp245
does not exist in pangeo_data.csv. Skipping.
IPSL-CM6A-LR-INCA_ssp24